# Import library

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Read dataset

Dataset obtained https://www.kaggle.com/datasets/utarasetyaw/data-set-destination-in-bali/

In [2]:
df_combination = pd.read_csv('./dataset/combination.csv')
df_combination.head()

,Place 1,Place 2,Place 3,Place 4,Place 5
0,Pantai Pasih Uug,Pantai Angel's Billabong,Pantai Kelingking,Pantai Crystal Bay,NaN
1,Pantai Diamond,Pantai Atuh,Bukit Teletubies,NaN,NaN
2,Tanjung Benoa,Pantai Waterblow,Pura Uluwatu,Pantai Pandawa,Pantai Jimbaran
3,Tari Barong dan Keris,Sari Merta,Pura Tirta Empul,Pantai Jimbaran,NaN
4,Tari Barong dan Keris,Sari Merta,Objek Wisata Desa Panglipuran,Pantai Jimbaran,NaN


## Dataset rating

In [3]:
df_rating = pd.read_csv('./dataset/rating.csv')
df_rating.head()

,User_Id,Place_Id,Nama Tempat,Place_Rating
0,1,2,Agung Bali Oleh-Oleh,3
1,1,7,Aloha Ubud Swing,5
2,1,8,Bali Bird Park,5
3,1,12,Big Garden Corner,2
4,1,13,Wanagiri Hidden Hills,5


In [4]:
df_rating.dtypes

User_Id          int64
Place_Id         int64
Nama Tempat     object
Place_Rating     int64
dtype: object

Get user count

In [5]:
user_count = df_rating['User_Id'].nunique()
user_count

100

In [6]:
df_rating['User_Id'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int64)

## Dataset tourism

In [7]:
df_tourism = pd.read_csv('./dataset/tourism.csv')
df_tourism.head()

,Place_Id,Place_Name,Description,Weekend Holiday Price,Weekday Price,Category,City,Rating,Alamat,Coordinate,Lat,Long,Gambar
0,1,Agrowisata Satria,Agrowisata Satria menawarkan 'rasa' Bali pada ...,50000,50000,Agrowisata,Gianyar,3.0,"Manukaya, Kec. Tampaksiring, Kabupaten Gianya...","-8.402228546131536, 115.32296693687739",-8.402229,115.322967,V
1,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,0,0,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali","-8.700234336021559, 115.176534407375",-8.700234,115.176534,V
2,3,Wisata Air Panas Toya Bungkah,Pemandian air panas di dekat gunung Batur (Pem...,70000,70000,Alam,Bangli,4.0,"Batur Tengah, Kec. Kintamani, Kabupaten Bangli...","-8.251298580809106, 115.39981444195475",-8.251299,115.399814,V
3,4,Air Terjun Aling-Aling,Keberadaan air terjun ini semakin mempercantik...,20000,20000,Alam,Buleleng,4.5,"Jl. Raya Desa Sambangan, Banjar, Sambangan, Ke...","-8.173073590748519, 115.10513024047802",-8.173074,115.105130,V
4,5,Air Terjun Tegenungan,Bosan dengan wisata alam pantai dan ingin meni...,20000,20000,Alam,Gianyar,4.0,"Jl. Ir. Sutami, Kemenuh, Kec. Sukawati, Kabupa...","-8.575191460385602, 115.28870189592831",-8.575191,115.288702,V


In [8]:
df_tourism.dtypes

Place_Id                   int64
Place_Name                object
Description               object
Weekend Holiday Price      int64
Weekday Price              int64
Category                  object
City                      object
Rating                   float64
Alamat                    object
Coordinate                object
Lat                      float64
Long                     float64
Gambar                    object
dtype: object

Get POI count

In [9]:
poi_count = df_rating['Place_Id'].nunique()
poi_count

75

In [10]:
np.sort(df_rating['Place_Id'].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

# Membuat matriks user-rating

In [11]:
matrix_user_rating = np.zeros((poi_count, user_count))

In [12]:
for i in range(0, poi_count):
    for j in range(0, user_count):
        try:
            matrix_user_rating[i][j] = df_rating[(df_rating['User_Id'] == i + 1 )& (df_rating['Place_Id'] == j + 1)].iloc[0]['Place_Rating']
        except Exception:
            continue

In [13]:
matrix_user_rating

array([[0., 3., 0., ..., 0., 0., 0.],
       [0., 3., 5., ..., 0., 0., 0.],
       [4., 5., 0., ..., 0., 0., 0.],
       ...,
       [0., 3., 0., ..., 0., 0., 0.],
       [4., 4., 5., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [14]:
df_user_rating = pd.DataFrame(matrix_user_rating,
                              index=range(1, poi_count + 1),
                              columns=range(1, user_count + 1))
df_user_rating.head()


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1,0.0,3.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,5.0,0.0,2.0,3.0,5.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,5.0,0.0,5.0,0.0,1.0,0.0,5.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_user_rating.to_csv('./dataset/user_rating.csv', index=False)

# Prediksi rating

## User-based Collaborative Filtering

In [16]:
def predict_ratings_user_based(rating_matrix):
    # Create a copy of the rating matrix
    predictions = rating_matrix.copy()

    # Calculate user similarity matrix using cosine similarity
    # Transpose the matrix because we want user similarity (users are columns)
    user_similarity = cosine_similarity(rating_matrix.T, rating_matrix.T)

    # For each item-user pair with zero rating
    for item in range(rating_matrix.shape[0]):
        for user in range(rating_matrix.shape[1]):
            if rating_matrix[item, user] == 0:
                # Get similar users (exclude the current user)
                similar_users = user_similarity[user]
                similar_users[user] = 0  # exclude self-similarity

                # Get ratings for the current item from other users
                item_ratings = rating_matrix[item]

                # Consider only non-zero ratings
                mask = item_ratings != 0

                # Calculate weighted average of ratings from similar users
                numerator = np.sum(similar_users * item_ratings * mask)
                denominator = np.sum(np.abs(similar_users) * mask)

                if denominator != 0:
                    predictions[item, user] = numerator / denominator
                else:
                    # If no similar users rated this item, use mean of non-zero ratings
                    non_zero_ratings = item_ratings[item_ratings != 0]
                    predictions[item, user] = np.mean(non_zero_ratings) if len(non_zero_ratings) > 0 else 0

    return predictions

# Apply the prediction
predicted_ratings = predict_ratings_user_based(matrix_user_rating)

# Create DataFrame for better visualization
df_user_based_predicted_ratings = pd.DataFrame(predicted_ratings,
                                  index=range(1, matrix_user_rating.shape[0] + 1),
                                  columns=range(1, matrix_user_rating.shape[1] + 1))

predicted_ub_cf_matrix = df_user_based_predicted_ratings.to_numpy()
predicted_ub_cf_matrix

array([[3.92380705, 3.        , 3.97565102, ..., 3.93333333, 3.93333333,
        3.93333333],
       [4.25528985, 3.        , 5.        , ..., 4.16666667, 4.16666667,
        4.16666667],
       [4.        , 5.        , 4.52479864, ..., 4.53333333, 4.53333333,
        4.53333333],
       ...,
       [3.88842673, 3.        , 3.84884551, ..., 3.93333333, 3.93333333,
        3.93333333],
       [4.        , 4.        , 5.        , ..., 3.76666667, 3.76666667,
        3.76666667],
       [1.        , 3.39816685, 3.42051977, ..., 3.5       , 3.5       ,
        3.5       ]])

## Item-based Collaborative Filtering

In [17]:
def predict_ratings_item_based(rating_matrix):
    # Create a copy of the rating matrix
    predictions = rating_matrix.copy()

    # Calculate item similarity matrix using cosine similarity
    # No need to transpose since rows are already items
    item_similarity = cosine_similarity(rating_matrix, rating_matrix)

    # For each item-user pair with zero rating
    for item in range(rating_matrix.shape[0]):
        for user in range(rating_matrix.shape[1]):
            if rating_matrix[item, user] == 0:
                # Get similar items (exclude the current item)
                similar_items = item_similarity[item]
                similar_items[item] = 0  # exclude self-similarity

                # Get user's ratings for all items
                user_ratings = rating_matrix[:, user]

                # Consider only non-zero ratings
                mask = user_ratings != 0

                # Calculate weighted average of ratings from similar items
                numerator = np.sum(similar_items * user_ratings * mask)
                denominator = np.sum(np.abs(similar_items) * mask)

                if denominator != 0:
                    predictions[item, user] = numerator / denominator
                else:
                    # If no similar items were rated by this user, use mean of non-zero ratings for the item
                    non_zero_ratings = rating_matrix[item][rating_matrix[item] != 0]
                    predictions[item, user] = np.mean(non_zero_ratings) if len(non_zero_ratings) > 0 else 0

    return predictions

# Apply the prediction
predicted_ratings = predict_ratings_item_based(matrix_user_rating)

# Create DataFrame for better visualization
df_item_based_predicted_ratings = pd.DataFrame(predicted_ratings,
                                  index=range(1, matrix_user_rating.shape[0] + 1),
                                  columns=range(1, matrix_user_rating.shape[1] + 1))
predicted_ib_cf_matrix = df_item_based_predicted_ratings.to_numpy()
predicted_ib_cf_matrix


array([[2.77027174, 3.        , 3.95796556, ..., 3.93333333, 3.93333333,
        3.93333333],
       [2.73873318, 3.        , 5.        , ..., 4.16666667, 4.16666667,
        4.16666667],
       [4.        , 5.        , 3.91862689, ..., 4.53333333, 4.53333333,
        4.53333333],
       ...,
       [2.78127436, 3.        , 3.93475525, ..., 3.93333333, 3.93333333,
        3.93333333],
       [4.        , 4.        , 5.        , ..., 3.76666667, 3.76666667,
        3.76666667],
       [1.        , 3.85808676, 3.99479862, ..., 3.5       , 3.5       ,
        3.5       ]])

## SVD

In [18]:
class MatrixFactorization:
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict missing ratings.

        Parameters:
        - R (np.array): User-item rating matrix with 0 for missing values
        - K (int): Number of latent factors
        - alpha (float): Learning rate
        - beta (float): Regularization parameter
        - iterations (int): Number of iterations for SGD
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent factor matrices with small random values
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Create a mask for observed ratings (non-zero entries)
        self.observed = self.R > 0

        # SGD optimization
        for it in range(self.iterations):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if self.observed[i, j]:
                        # Compute error of prediction
                        prediction = self.predict_single(i, j)
                        e_ij = self.R[i, j] - prediction

                        # Update latent factors with gradient descent
                        self.P[i, :] += self.alpha * (e_ij * self.Q[j, :] - self.beta * self.P[i, :])
                        self.Q[j, :] += self.alpha * (e_ij * self.P[i, :] - self.beta * self.Q[j, :])

            # Optionally, compute total error to monitor convergence
            error = self.compute_error()
            if (it + 1) % 10 == 0 or it == 0:
                print(f"Iteration {it+1}/{self.iterations}, error: {error:.4f}")

    def predict_single(self, i, j):
        """Predict rating of user i for item j."""
        return np.dot(self.P[i, :], self.Q[j, :].T)

    def full_matrix(self):
        """Reconstruct the full rating matrix with predicted values."""
        return np.dot(self.P, self.Q.T)

    def compute_error(self):
        """Compute total squared error on observed ratings with regularization."""
        prediction = self.full_matrix()
        error = 0
        for i in range(self.num_users):
            for j in range(self.num_items):
                if self.observed[i, j]:
                    error += (self.R[i, j] - prediction[i, j]) ** 2
        # Add regularization terms
        error += self.beta * (np.sum(np.square(self.P)) + np.sum(np.square(self.Q)))
        return error

mf = MatrixFactorization(matrix_user_rating, K=3, alpha=0.01, beta=0.01, iterations=100)
mf.train()

predicted_svd_matrix = mf.full_matrix()
predicted_svd_matrix = np.clip(predicted_svd_matrix, 0, 5)  # Clip ratings to [0, 5]

predicted_svd_matrix

Iteration 1/100, error: 40411.0511
Iteration 10/100, error: 1859.2313
Iteration 20/100, error: 1742.5116
Iteration 30/100, error: 1667.5425
Iteration 40/100, error: 1613.4014
Iteration 50/100, error: 1569.5336
Iteration 60/100, error: 1533.9825
Iteration 70/100, error: 1506.9904
Iteration 80/100, error: 1487.1942
Iteration 90/100, error: 1472.2602
Iteration 100/100, error: 1460.5386


array([[1.70406113, 3.3657102 , 3.70226911, ..., 0.        , 0.16402489,
        1.40566533],
       [2.32217312, 4.03374077, 4.23719002, ..., 0.        , 0.59415891,
        1.40127655],
       [2.5376784 , 4.30913709, 4.49238445, ..., 0.        , 0.63762627,
        1.48946578],
       ...,
       [4.0245523 , 3.38797297, 2.34240353, ..., 0.08246914, 0.31001832,
        1.09418116],
       [2.09845676, 3.45532903, 3.55374096, ..., 0.        , 0.63837417,
        1.10630907],
       [2.67036412, 2.56318848, 2.0463661 , ..., 0.        , 0.        ,
        1.25845633]])

## Alternating Least Square (ALS)

In [19]:
class ALS:
    def __init__(self, R, K=10, lambda_reg=0.1, iterations=20):
        """
        ALS matrix factorization for explicit feedback.

        Parameters:
        - R (np.array): User-item rating matrix with 0 for missing ratings
        - K (int): Number of latent factors
        - lambda_reg (float): Regularization parameter
        - iterations (int): Number of ALS iterations
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.lambda_reg = lambda_reg
        self.iterations = iterations

        # Initialize user and item latent factor matrices with random values
        self.U = np.random.rand(self.num_users, K)
        self.V = np.random.rand(self.num_items, K)

        # Mask for observed ratings
        self.observed = R > 0

    def train(self):
        for it in range(self.iterations):
            # Fix V, update U
            for i in range(self.num_users):
                V_i = self.V[self.observed[i, :], :]
                R_i = self.R[i, self.observed[i, :]]
                if V_i.shape[0] > 0:
                    A = V_i.T @ V_i + self.lambda_reg * np.eye(self.K)
                    b = V_i.T @ R_i
                    self.U[i, :] = np.linalg.solve(A, b)

            # Fix U, update V
            for j in range(self.num_items):
                U_j = self.U[self.observed[:, j], :]
                R_j = self.R[self.observed[:, j], j]
                if U_j.shape[0] > 0:
                    A = U_j.T @ U_j + self.lambda_reg * np.eye(self.K)
                    b = U_j.T @ R_j
                    self.V[j, :] = np.linalg.solve(A, b)

            error = self.compute_error()
            print(f"Iteration {it+1}/{self.iterations}, error: {error:.4f}")

    def predict(self):
        """Reconstruct the full rating matrix."""
        return self.U @ self.V.T

    def compute_error(self):
        """Compute squared error on observed ratings with regularization."""
        prediction = self.predict()
        error = 0
        for i in range(self.num_users):
            for j in range(self.num_items):
                if self.observed[i, j]:
                    error += (self.R[i, j] - prediction[i, j]) ** 2
        # Add regularization terms
        error += self.lambda_reg * (np.sum(np.square(self.U)) + np.sum(np.square(self.V)))
        return error

# mf = ALS(matrix_user_rating, K=3, alpha=0.01, beta=0.01, iterations=100)
als = ALS(matrix_user_rating, K=3, lambda_reg=0.1, iterations=30)
als.train()

predicted_als_matrix = als.predict()
predicted_als_matrix = np.clip(predicted_als_matrix, 0, 5)

predicted_als_matrix = np.round(predicted_als_matrix, 2)
predicted_als_matrix


Iteration 1/30, error: 1925.6393
Iteration 2/30, error: 1661.3836
Iteration 3/30, error: 1597.4863
Iteration 4/30, error: 1567.8665
Iteration 5/30, error: 1548.4354
Iteration 6/30, error: 1535.5166
Iteration 7/30, error: 1527.0504
Iteration 8/30, error: 1521.2784
Iteration 9/30, error: 1517.0499
Iteration 10/30, error: 1513.7119
Iteration 11/30, error: 1510.9178
Iteration 12/30, error: 1508.4706
Iteration 13/30, error: 1506.2349
Iteration 14/30, error: 1504.0967
Iteration 15/30, error: 1501.9424
Iteration 16/30, error: 1499.6426
Iteration 17/30, error: 1497.0277
Iteration 18/30, error: 1493.8511
Iteration 19/30, error: 1489.7332
Iteration 20/30, error: 1484.1716
Iteration 21/30, error: 1477.0263
Iteration 22/30, error: 1469.6439
Iteration 23/30, error: 1464.0607
Iteration 24/30, error: 1460.5766
Iteration 25/30, error: 1458.3935
Iteration 26/30, error: 1456.8710
Iteration 27/30, error: 1455.6806
Iteration 28/30, error: 1454.6684
Iteration 29/30, error: 1453.7608
Iteration 30/30, error:

array([[1.67, 3.01, 4.59, ..., 2.49, 3.46, 4.45],
       [4.66, 2.98, 4.67, ..., 2.43, 2.93, 4.72],
       [3.35, 4.5 , 3.87, ..., 3.28, 3.55, 5.  ],
       ...,
       [5.  , 3.14, 3.33, ..., 2.33, 2.26, 4.48],
       [4.45, 2.73, 4.02, ..., 2.19, 2.53, 4.26],
       [1.24, 0.  , 5.  , ..., 0.  , 2.93, 1.13]])

## PMF

In [20]:
class PMF:
    def __init__(self, R, K=10, lambda_u=0.1, lambda_v=0.1, learning_rate=0.005, iterations=100):
        """
        Probabilistic Matrix Factorization using gradient descent.

        Parameters:
        - R (np.array): User-item rating matrix with 0 for missing ratings
        - K (int): Number of latent factors
        - lambda_u (float): Regularization for user factors
        - lambda_v (float): Regularization for item factors
        - learning_rate (float): Learning rate for gradient descent
        - iterations (int): Number of iterations for training
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.lambda_u = lambda_u
        self.lambda_v = lambda_v
        self.learning_rate = learning_rate
        self.iterations = iterations

        # Initialize latent factors with small random values
        self.U = 0.1 * np.random.randn(self.num_users, K)
        self.V = 0.1 * np.random.randn(self.num_items, K)

        # Mask for observed ratings
        self.observed = R > 0

    def train(self):
        for it in range(self.iterations):
            self.gradient_descent_step()
            error = self.compute_error()
            if (it + 1) % 10 == 0 or it == 0:
                print(f"Iteration {it+1}/{self.iterations}, error: {error:.4f}")

    def gradient_descent_step(self):
        for i in range(self.num_users):
            for j in range(self.num_items):
                if self.observed[i, j]:
                    prediction = self.U[i, :].dot(self.V[j, :].T)
                    e_ij = self.R[i, j] - prediction

                    # Update user and item latent factors
                    self.U[i, :] += self.learning_rate * (e_ij * self.V[j, :] - self.lambda_u * self.U[i, :])
                    self.V[j, :] += self.learning_rate * (e_ij * self.U[i, :] - self.lambda_v * self.V[j, :])

    def predict(self):
        """Reconstruct the full rating matrix."""
        return self.U.dot(self.V.T)

    def compute_error(self):
        """Compute regularized squared error on observed ratings."""
        prediction = self.predict()
        error = 0
        for i in range(self.num_users):
            for j in range(self.num_items):
                if self.observed[i, j]:
                    error += (self.R[i, j] - prediction[i, j]) ** 2
        # Add regularization terms
        error += self.lambda_u * np.sum(self.U ** 2) + self.lambda_v * np.sum(self.V ** 2)
        return error

# Example user-item matrix (0 means missing rating)
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

pmf = PMF(matrix_user_rating, K=3, lambda_u=0.1, lambda_v=0.1, learning_rate=0.005, iterations=100)
pmf.train()

predicted_pmf_matrix = pmf.predict()
# Clip predicted ratings to [0, 5]
predicted_pmf_matrix = np.clip(predicted_pmf_matrix, 0, 5)
predicted_pmf_matrix = np.round(predicted_pmf_matrix, 2)
predicted_pmf_matrix

Iteration 1/100, error: 40956.9545
Iteration 10/100, error: 2449.2223
Iteration 20/100, error: 2008.6438
Iteration 30/100, error: 1991.0020
Iteration 40/100, error: 1963.9998
Iteration 50/100, error: 1923.6401
Iteration 60/100, error: 1870.3659
Iteration 70/100, error: 1811.0516
Iteration 80/100, error: 1754.6357
Iteration 90/100, error: 1706.2245
Iteration 100/100, error: 1667.1305


array([[1.59, 3.38, 3.99, ..., 0.12, 0.06, 0.  ],
       [2.36, 3.83, 4.2 , ..., 0.24, 0.  , 0.  ],
       [2.9 , 4.22, 4.11, ..., 0.22, 0.  , 0.  ],
       ...,
       [1.93, 3.42, 3.56, ..., 0.09, 0.05, 0.  ],
       [2.08, 3.25, 4.02, ..., 0.37, 0.  , 0.  ],
       [1.05, 2.94, 3.25, ..., 0.  , 0.14, 0.  ]])

# Evaluation

In [21]:
def evaluate_predictions(true_ratings, _predicted_ratings, mask):
    """
    Evaluate predictions using various metrics

    Parameters:
    - true_ratings: actual rating matrix
    - _predicted_ratings: predicted rating matrix
    - mask: boolean matrix indicating which entries to evaluate (True for test entries)

    Returns:
    Dictionary containing different evaluation metrics
    """
    # Get only the predictions we want to evaluate (where mask is True)
    y_true = true_ratings[mask]
    y_pred = _predicted_ratings[mask]

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    return {
        'RMSE': rmse,
        'MAE': mae,
        'MSE': mse
    }

In [22]:
def train_test_split_matrix(rating_matrix, test_ratio=0.2):
    """
    Split the rating matrix into training and test sets

    Parameters:
    - rating_matrix: The original rating matrix
    - test_ratio: Proportion of non-zero entries to use for testing

    Returns:
    - train_matrix: Training matrix with test entries as 0
    - test_mask: Boolean matrix indicating test entries
    """
    # Create copy of the original matrix
    train_matrix = rating_matrix.copy()

    # Create mask for non-zero entries
    non_zero_mask = rating_matrix != 0

    # Create test mask initialized with False
    test_mask = np.zeros_like(rating_matrix, dtype=bool)

    # Get indices of non-zero entries
    non_zero_indices = np.where(non_zero_mask)
    n_non_zero = len(non_zero_indices[0])

    # Randomly select indices for test set
    test_size = int(n_non_zero * test_ratio)
    test_indices = np.random.choice(n_non_zero, test_size, replace=False)

    # Mark selected entries as test entries
    test_mask[non_zero_indices[0][test_indices],
              non_zero_indices[1][test_indices]] = True

    # Set test entries to 0 in training matrix
    train_matrix[test_mask] = 0

    return train_matrix, test_mask

In [23]:
# First split the data
train_matrix, test_mask = train_test_split_matrix(matrix_user_rating)

# For each method, train on train_matrix and evaluate on test set
# User-based CF
predicted_ub = predict_ratings_user_based(train_matrix)
ub_metrics = evaluate_predictions(matrix_user_rating, predicted_ub, test_mask)

# Item-based CF
predicted_ib = predict_ratings_item_based(train_matrix)
ib_metrics = evaluate_predictions(matrix_user_rating, predicted_ib, test_mask)

# SVD
mf = MatrixFactorization(train_matrix, K=3, alpha=0.01, beta=0.01, iterations=100)
mf.train()
predicted_svd = mf.full_matrix()
svd_metrics = evaluate_predictions(matrix_user_rating, predicted_svd, test_mask)

# ALS
als = ALS(train_matrix, K=10, lambda_reg=0.1, iterations=20)
als.train()
predicted_als = als.predict()
als_metrics = evaluate_predictions(matrix_user_rating, predicted_als, test_mask)

# PMF
pmf = PMF(matrix_user_rating, K=3, lambda_u=0.1, lambda_v=0.1, learning_rate=0.005, iterations=100)
pmf.train()
predicted_pmf = als.predict()
pmf_metrics = evaluate_predictions(matrix_user_rating, predicted_pmf, test_mask)

# Print results
methods = ['User-based CF', 'Item-based CF', 'SVD', 'ALS', 'PMF']
metrics = [ub_metrics, ib_metrics, svd_metrics, als_metrics, pmf_metrics]

print("\nEvaluation Results:")
print("-" * 50)
for method, metric in zip(methods, metrics):
    print(f"\n{method}:")
    for metric_name, value in metric.items():
        print(f"{metric_name}: {value:.4f}")

Iteration 1/100, error: 32057.6333
Iteration 10/100, error: 1405.4277
Iteration 20/100, error: 1266.2060
Iteration 30/100, error: 1157.9053
Iteration 40/100, error: 1089.2558
Iteration 50/100, error: 1044.7988
Iteration 60/100, error: 1015.0444
Iteration 70/100, error: 995.1739
Iteration 80/100, error: 981.7320
Iteration 90/100, error: 972.2409
Iteration 100/100, error: 965.0170
Iteration 1/20, error: 765.8105
Iteration 2/20, error: 445.3199
Iteration 3/20, error: 358.5873
Iteration 4/20, error: 315.3870
Iteration 5/20, error: 289.1581
Iteration 6/20, error: 271.9549
Iteration 7/20, error: 259.2445
Iteration 8/20, error: 249.3587
Iteration 9/20, error: 241.6022
Iteration 10/20, error: 235.3894
Iteration 11/20, error: 230.2638
Iteration 12/20, error: 225.9402
Iteration 13/20, error: 222.2396
Iteration 14/20, error: 219.0412
Iteration 15/20, error: 216.2629
Iteration 16/20, error: 213.8469
Iteration 17/20, error: 211.7477
Iteration 18/20, error: 209.9245
Iteration 19/20, error: 208.3393


# Export data

In [24]:
results = [predicted_ub_cf_matrix, predicted_ib_cf_matrix, predicted_svd_matrix, predicted_als_matrix, predicted_pmf_matrix]
for method, result in zip(methods, results):
    df_user_rating = pd.DataFrame(result,
                                  index=range(1, poi_count + 1),
                                  columns=range(1, user_count + 1))
    df_user_rating.to_csv(f'./dataset/user_rating_{method}.csv', index=False)